##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# TensorFlow Graph Optimization

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/not_a_real_link"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/tools/templates/notebook.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/tools/templates/notebook.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/tools/templates/notebook.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

## Overview

TensorFlow optimizes the TensorFlow graph (built by `tf.function`) before executing operations.
This makes the TensorFlow graph more efficiently and less peak memory usage.
As most of optimizations are enabled automatically, you don't need to know what optimizations are perfomred under the TensorFlow.

On the other hand, TensorFlow provides the way for the advanced TensorFlow users to enable/disable the optimization by using `tf.config.optimizer.set_experimental_options()`.

## Setup

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x  #gpu
except Exception:
  pass
import tensorflow as tf

## Graph Optimization performed in TensorFlow

TensorFlow performs many optimizations to the TensorFlow Graph such as Constant Foling, Memory Optimizer, etc...  
All optimizations are listed in [tf.config.optimizer.set_experimental_options()](https://www.tensorflow.org/api_docs/python/tf/config/optimizer/set_experimental_options) API document page.
Below optimizations are typical optimizations performed in TensorFlow.

* **Constant Folding** evaluates the operations whose input tensors are all constant.
* **Memory Optimizer** analyzes the TensorFlow graph to inspect the peak memory usage at each operations. To reduce a peak memory usage, Memory Optimizer inserts CPU-GPU memory copy operations for swapping GPU memory to CPU, or rewrites TensorFlow graph for the recomputation.
* **Layout Optimizer** inserts Transpose operation to change data-format in order to execute data-format depended operation more efficiently on GPU.
* **Arithmentic Optimizer** is an optimization to rewrite the TensorFlow graph using mathematical equivalence relation.
* **Debug Stripper** strips debug operations such as Assert.

More detail information about the graph optimization can be found in [this slide](http://web.stanford.edu/class/cs245/slides/TFGraphOptimizationsStanford.pdf).

## Checking Enabled Graph Optimization

Enabled optimization configuration can be shown by calling `tf.config.optimizer.get_experimental_options()`.

Below simple code shows the default optimization configuration.

In [ ]:
import tensorflow as tf

tf.config.optimizer.get_experimental_options()

## Inspect the Optimized Graph

The optimized graphs can be inspectable with using TensorBoard.

Let us look at the case of "Constant Folding" optimization.
"Constant Folding" is enabled by default, so you will see the effects of the constant propagations without any procedures.

To output TensorBoard summary data, you must call `tf.summary.trace_on()` before executing the graph.
All profile and graph data (includes optimized graphs) are stored in memory.
After the graph execution, TensorBoard summary data can be output by calling `tf.summary.trace_export()`.

In [ ]:
import tensorflow as tf

@tf.function
def simple_func(arg):
    a = tf.constant(7.9)
    b = tf.constant(6.3)
    c = arg + a
    d = a * b
    ret = c + d
    
    return ret

# Enable tracing data to inspect the optimized graph.
writer = tf.summary.create_file_writer("summary")
tf.summary.trace_on(graph=True, profiler=True)

arg = tf.constant(8.9)
print(simple_func(arg))

# Output traced data as TensorBoard summary data.
with writer.as_default():
    tf.summary.trace_export("summary", step=0, profiler_outdir="summary")

# Disable tracing data.
tf.summary.trace_off()

Then, launch TensorBoard to inspect the optimized graph.

The optimized graph is inspectable via Profile graph.
The first image shows the user defined graph, and the second image shows the optimized graph.
We can notice that mul node and it's input (Const) nodes are missing in the optimized graph.
This indicates that Constant Folding works well against the user defined graph.

![User Defined Graph](user_defined_graph.png)

![Optimized Graph](optimized_graph.png)

## Enable/Disable Graph Optimization

Most of graph optimizations can be enabled/disabled by calling `tf.config.optimizer.set_experimental_options()`.

Below simple code shows how to enable/disable the optimization.

* Disable "Debug Stripper"

**Debug Stripper** which is disabled by default, strips the operations used for the debug purpose (Assert, CheckNumerics, Print).
Below code will raise an exception at `tf.Assert`.

In [ ]:
import tensorflow as tf

@tf.function
def assert_func():
    a = tf.constant(1.2)
    computation_graph = tf.Assert(tf.less_equal(a, 1.0), [a])   # Will raise an "InvalidArgumentError" exception.
    return a

print(assert_func())

If you make Debug Stripper enabled, below code does not raise any exceptions.

In [ ]:
import tensorflow as tf

# Enable "Debug Stripper".
tf.config.optimizer.set_experimental_options({'debug_stripper': True})

@tf.function
def assert_func():
    a = tf.constant(1.2)
    computation_graph = tf.Assert(tf.less_equal(a, 1.0), [a])   # No exceptions are raised.
    return a

print(assert_func())

* Disable All Graph Optimizations

All Graph Optimization can be disabled when `'disable_meta_optimizer': False` is passed to `tf.config.optimizer.set_experimental_options()`.

At first, let us check the optimized graph with default optimization configuration.
Below code builds the graph which has a seqence of Transpose operations.

In [ ]:
import tensorflow as tf
import numpy as np

@tf.function
def optimized(arg):
    a = arg * 2

    # Will be simplified by Arithmetic Optimizer.
    b = tf.transpose(a, perm=[1, 0])
    ret = tf.transpose(b, perm=[1, 0])

    return ret

writer = tf.summary.create_file_writer("summary")
tf.summary.trace_on(graph=True, profiler=True)

arg = tf.constant(np.random.normal(size=(30, 40)))
optimized(arg)

with writer.as_default():
    tf.summary.trace_export("summary", step=0, profiler_outdir="summary")

tf.summary.trace_off()

As you can see in TensorBoard, Transpose operation was erased by Arithmetic Optimizer to simplify Transpose operation into NoOp.

![Meta Optimizer Enabled](meta_optimizer_enabled.png)

Next, let us check the graph with `'disable_meta_optimizer'` on.

In [ ]:
import tensorflow as tf
import numpy as np

# Disable all graph optimizations.
tf.config.optimizer.set_experimental_options({'disable_meta_optimizer': True})

@tf.function
def not_optimized(arg):
    a = arg * 2
    b = tf.transpose(a, perm=[1, 0])
    ret = tf.transpose(b, perm=[1, 0])

    return ret

writer = tf.summary.create_file_writer("summary")
tf.summary.trace_on(graph=True, profiler=True)

arg = tf.constant(np.random.normal(size=(30, 40)))
not_optimized(arg)

with writer.as_default():
    tf.summary.trace_export("summary", step=0, profiler_outdir="summary")

tf.summary.trace_off()

In this graph, Transpose operations are remained in the graph, which indicates Arithmetic Optimizer was disabled.

![Meta Optimizer Disabled](meta_optimizer_disabled.png)